In [5]:
import pickle
with open('Unnamed Titration.pkl', 'rb') as fh:
    titration = pickle.load(fh)


In [1]:
import io
from ipyfileupload.widgets import DirectoryUploadWidget
def _upload():

    _upload_widget = DirectoryUploadWidget()

    def _cb(change):
        for name, file in change['owner'].files.items():
            decoded = io.StringIO(file.decode('utf-8'))
            print('Uploaded `{}` ({:.2f} kB)'.format(
                name, len(decoded.read()) / 2 **10))

    _upload_widget.observe(_cb, names='base64_files')
    display(_upload_widget)
    return _upload_widget

widget = _upload()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [2]:
from classes.ipywidgets import *
from classes.Titration import Titration, TitrationCLI
from ipyfileupload.widgets import DirectoryUploadWidget

TitrationWidget.titration = Titration()
DirectoryUploadWidget()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [3]:


view = TitrationFilesView()
upload = TitrationDirUploader()
upload.add_observer(view)
display(
    upload,
    view
)

TitrationDirUploader(label='Upload titration directory...', style=ButtonStyle(), _dom_classes=('widget_item', 'btn-group'))

TitrationFilesView(children=(VBox(children=(Label(value='Data files', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(_dom_classes=('panel-body',))), layout=Layout(width='200px'), _dom_classes=('panel', 'panel-default'))

In [4]:
startParams = StartParamContainer()
protocole = ProtocoleContainer()

upload.add_observer(startParams)
upload.add_observer(protocole)

display(startParams, protocole)

AttributeError: 'dict' object has no attribute 'titration'